# Generador de Tablas de Verdad en LaTex / LaTex Truth Tables Generator

A tree is used to represent the structure of a logical proposition. The buit methods take strings as input. The following operators are supported:

- Negation `¬` 
- Conjunction `^`
- Disjunction `|`
- Conditional `->`
- Biconditional `<->`

Parentheses must be used to avoid certain types of ambiguity. For example, $a\wedge b\implies c$ must be written as `"(a^b)->c"`. Also, all variables must be letters of the alphabet.

The method that is used to create the truth tables is `texTT`.

`texTT` takes the following arguments:
```
    texTT(filename, str_expr, standalone)
```
Where:
- `filename` is the name of the file to create.
- `str_expr` is either a string with the logical expression to calculate or a list of such strings (if there are multiple tables to generate).
- `standalone` is a variable of type `bool` that specifies whether or not to include packages and the `document` environment in the file.
It creates de file in `./output`.

---

Se utliza un árbol para representar la la estructura de una proposición lógica. Los métodos incluidos toman cadenas de caracteres como entrada. Están considerados los siguientes operadores:

- Negación `¬` 
- Conjunción `^`
- Disyunción `|`
- Condicional `->`
- Bicondicional `<->`

Se deben usar paréntesis para evitar ambigüedades. Por ejemplo, $a\wedge b\implies c$ debe escribirse como `"(a^b)->c"`. También, las variables deben ser letras del alfabeto.

El método utilizado para crear las tablas es `texTT`.

`texTT` toma los siguientes argumentos:
```
    texTT(filename, str_expr, standalone)
```
Where:
- `filename` es el nombre del archivo que se va a crear.
- `str_expr` es ya sea una cadena con una expresión lógica, o una lista con cadenas de esa forma.
- `standalone` es una variable de tipo `bool` que especifica si hay que incluir los paquetes y el ambiente `document` en el archivo.
It creates de file in `./output`.


## Clases / Classes

In [1]:
import string
import os
import numpy as np

In [2]:
class Node:
    def __init__(self, data, *args):
        self.data = data
        self.children = []
        self.name = ""
        self.classname = "Node"
        self.complete = True
        self.noperator = 0
        
        if len(args)>0:
            self.name=args[0]
    
    def __str__(self):
        s = self.classname[:1] + "("+str(self.data)
        if self.name!="":
            s += ", \"" + self.name +"\""
        return s + ")"
    def __repr__(self):
        s = self.classname + "("+str(self.data)
        if self.name!="":
            s += ", \"" + self.name +"\""
        return s + ")"
    
    def __eq__(self, other):
        return self.name == other.name
    
    def printTreeNodes(self):
        if self.noperator == 0: #variable
            print(self)
        elif self.noperator == 1: #Operador unario
            print(self)
            self.children[0].printTreeNodes()
        elif self.noperator == 2: #Operador binario
            self.children[0].printTreeNodes()
            print(self)
            self.children[1].printTreeNodes()
        else: #Paréntesis
            print(self)
            self.children[0].printTreeNodes()
            print(")")
            
    def __stringExpressionRec__(self, s):
        cadena = s
        if self.noperator == 0: #variable
            cadena += self.name
        elif self.noperator == 1: #Operador unario
            cadena += self.name
            cadena += self.children[0].__stringExpressionRec__(s)
        elif self.noperator == 2: #Operador binario
            cadena += self.children[0].__stringExpressionRec__(s)
            cadena += self.name
            cadena += self.children[1].__stringExpressionRec__(s)
        else: #Paréntesis
            cadena += "("
            cadena += self.children[0].__stringExpressionRec__(s)
            cadena += ")"
        return cadena
    def stringExpression(self):
        return self.__stringExpressionRec__("")
    def printExpression(self):
        print(self.stringExpression())
    
    def setValue(self, value):
        self.data = value

    def isComplete(self):
        return self.complete
    
    def setComplete(self):
        self.complete = True
        
    def insert(self, N):
        self.children.append(N)
    
    def function(self):
        return self.data

In [3]:
class Parentesis(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Parentesis"
        self.complete = False
        self.noperator = -1
    
    def function(self):
        return self.children[0].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node (Parenthesis).")
        super().insert(N)
        if len(self.children)>=1:
            self.setComplete()

In [4]:
class Conjuncion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Conjuncion"
        self.name = "^"
        self.complete = False
        self.noperator = 2
    
    def function(self):
        return self.children[0].function() and self.children[1].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node (Conjunction).")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()

In [5]:
class Disyuncion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Disyuncion"
        self.name = "|"
        self.complete = False
        self.noperator = 2
        
    def function(self):
        return self.children[0].function() or self.children[1].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node (Disjunction).")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()

In [6]:
class Negacion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Negacion"
        self.name = "¬"
        self.complete = False
        self.noperator = 1
    
    def function(self):
        return not self.children[0].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node (Negation).")
        super().insert(N)
        if len(self.children)>=1:
            self.setComplete()

In [7]:
class Implicacion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Implicacion"
        self.name = "->"
        self.complete = False
        self.noperator = 2
    
    def function(self):
        if self.children[0].function()==True and self.children[1].function()==False:
            return False
        else:
            return True

    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node (Implication).")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()    

In [8]:
class DobleImplicacion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "DobleImplicacion"
        self.name = "<->"
        self.complete = False
        self.noperator = 2
    
    def function(self):
        return self.children[0].function()==self.children[1].function()

    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node (DoubleImplication).")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()    

## Functions

In [9]:
def string2Proposition(str_prop):
    Letras = string.ascii_lowercase + string.ascii_uppercase
    valid_chars = "|^¬ <->()"+Letras
    Variables = []
    node_stack = []
    j = None
    i=0
    while i < len(str_prop):        
        if str_prop[i] == ' ':
            i+=1
            continue
        
        #Agregar el nodo al stack, sea cual sea
        if str_prop[i]=='(':
            node_stack.append(Parentesis(None))
            i+=1
            continue
        
        if str_prop[i] in Letras:
            if str_prop[i] not in map(lambda x: x.name, Variables):
                Variables.append(Node(None, str_prop[i]))
            j = Variables.index(Node(None, str_prop[i]))   
            node_stack.append(Variables[j])
        
        if str_prop[i] == '¬':
            node_stack.append(Negacion(None))
            
        if str_prop[i] == '^':
            if len(str_prop)==1 or not node_stack[-1].isComplete():
                print("String error 1")
                return None
            node_stack.append(Conjuncion(None))
    
        if str_prop[i] == '|':
            if len(str_prop)==1 or not node_stack[-1].isComplete():
                print("String error 2")
                return None
            node_stack.append(Disyuncion(None))
        
        if str_prop[i] == '-':
            if (i >= len(str_prop)-1) or str_prop[i+1]!='>':
                print("String error 3")
                return None
            node_stack.append(Implicacion(None))
            i+=1 #Porque este operador es de dos caracteres
        
        if str_prop[i] == '<':
            if (i >= len(str_prop)-2) or str_prop[(i+1):(i+1+2)]!="->":
                print("String error 4")
                return None
            node_stack.append(DobleImplicacion(None))
            i+=2 #Porque este operador es de tres caracteres
        
        #Caso especial: Cierre de paréntesis
        if str_prop[i] == ')':
            #Asegurar que no haya habido algún error con la cadena
            if type(node_stack[-2])!= Parentesis:
                print("String error 5")
                return None
            #Agregar el contenido del paréntesis dentro del nodo paréntesis
            node_stack[-2].insert(node_stack[-1])
            del node_stack[-1]        
        
        #Decide what to do with the node that has been added to the stack
        #Binary operators:
            #Binary operators need to be linked with the previous node
        if node_stack[-1].noperator == 2:
            node_stack[-1].insert(node_stack[-2])
            del node_stack[-2]

        #Loop to insert completed nodes into previous nodes
        if node_stack[-1].isComplete():
            repeat = True
            while repeat:
                if (len(node_stack)==1) or (type(node_stack[-2])==Parentesis):#Cases in which the completed node doesn't need to be inserted in another node
                    repeat = False
                else: #The node needs to be inserted into another node
                    repeat = True
                    node_stack[-2].insert(node_stack[-1])
                    del node_stack[-1]
        i+=1

    return (Variables, node_stack[0])

In [10]:
def recursiveTree2Lists(node, funcs, exprs):
    if node.noperator == 2:
        recursiveTree2Lists(node.children[0], funcs, exprs)
        recursiveTree2Lists(node.children[1], funcs, exprs)
        funcs.append(node.function)
        exprs.append(node.stringExpression())
    elif node.noperator == 1:
        funcs.append(node.function)
        exprs.append(node.stringExpression())
        recursiveTree2Lists(node.children[0], funcs, exprs)
    elif type(node)==Parentesis:
        recursiveTree2Lists(node.children[0], funcs, exprs)
    else: #Variables
        pass
    
        

In [11]:
def expressionLists(exp_tree):
    
    func_list = []
    expr_list = []
    recursiveTree2Lists(exp_tree, func_list, expr_list)
    
    return (func_list, expr_list)

In [12]:
def texHeader(file):
    file.write("\\documentclass[12pt]{article}\r\n\\usepackage[utf8]{inputenc}\r\n\\pagestyle{plain}\r\n\\usepackage[cm]{fullpage}\r\n\\usepackage{amsmath}\r\n\\usepackage{float}\r\n\\usepackage{array}\r\n\\usepackage{graphicx}\r\\usepackage{longtable}\r\n")

In [13]:
def laTexTrueFalse(value):
    return "$V$" if value else "$F$"

In [14]:
#Función para crear un tabular en latex con una tabla de verdad
def texTT(filename, str_expr, standalone):
    #Path
    path = os.getcwd()+"/output/"
    os.makedirs(path, exist_ok = True)
    
    #Open file
    file = open(path+filename, "w")
    
    #Obtain expression_list
    if type(str_expr) == str:
        input_exprs = [str_expr]
    elif type(str_expr) == list:
        input_exprs = str_expr
    else:
        print("Wrong argument type for 'str_expr'")
        return None
    
    #Latex pre-table(s)
    if standalone:
        texHeader(file)
        file.write("\\begin{document}\n")
    
    for e in input_exprs:
        
        #Obtener funciones y nombres de columnas
        (variables, tree) = string2Proposition(e)
        (func_list, expr_list) = expressionLists(tree)
        #Add variables to the left of the table
        func_list = list(map(lambda x:x.function, variables)) + func_list
        expr_list = list(map(lambda x:x.stringExpression(), variables)) + expr_list
        #Calculate each expressions width
        expr_widths = [len(e) for e in expr_list]
        #Convert expr_list to latex formatting
        expr_list = list(map(convertStr2LaTex, expr_list))
        
        #Begin Table
        total_expr_length = sum(expr_widths)
        columnformat="{|"
        for i in range(len(expr_list)):
            columnformat+=">{\\centering\\arraybackslash}p{" + str(np.floor(0.7*expr_widths[i]/total_expr_length*10**4)/(10**4))+"\linewidth}|"
        columnformat+="}"
        file.write("\t\\begin{center}\n")#\\begin{table}[ht]\n")
        #file.write("\t\\centering\n")
        #file.write("\t\\scalebox{0.8}{\n")
        file.write("\t\\begin{longtable}"+columnformat+"\n")

        #Tabular Header:
        file.write("\t\t\\hline\n\t\t")
        for i in range(len(expr_list)):
            file.write(expr_list[i])
            if i != len(expr_list)-1:
                file.write(" & ")

        file.write("\\\\ \n\t\t\\hline\n")

        #Main Truth Table cycle
        N = len(variables)
        for i in range(2**N):
            valores = [bool(int(x)) for x in bin(i)[2:].zfill(N)]
            for j in range(N):
                variables[j].setValue(valores[j])
            file.write("\t\t")
            for j in range(len(func_list)):
                file.write(laTexTrueFalse(func_list[j]()))
                if j<len(func_list)-1:
                    file.write(" & ")
            file.write("\\\\ \n\t\t\\hline\n")
        #End Table
        #file.write("\t\\end{tabular}\n")
        #file.write("\t}\n")
        file.write("\t\\end{longtable}\\end{center}\n")
    
    #End document if needed
    if standalone:
        file.write("\\end{document}\n")
    
    #Close file
    file.close()
    #Print what has been created
    print("The file '"+path+filename+"' has been created" )
    

In [15]:
def convertStr2LaTex(str_prop):
    Letras = string.ascii_lowercase + string.ascii_uppercase
    valid_chars = "|^¬ <->()"+Letras
    
    str2 = "{\small $"
    i=0
    while(i<len(str_prop)):
        if str_prop[i] not in valid_chars:
            print("Not suported character:", str_expr[i])
            return None
        
        if str_prop[i]=='(':
            str2 += "("
        elif str_prop[i] in Letras:
            str2 += str_prop[i]
        elif str_prop[i] == '¬':
            str2 += r"\neg "
        elif str_prop[i] == '^':
            str2 += r"\wedge "
        elif str_prop[i] == '|':
            str2 += r"\vee "
        elif str_prop[i] == '-':
            if (i >= len(str_prop)-1) or str_prop[i+1]!='>':
                print("String error 3")
                return None
            str2 += r"\Rightarrow "#r"\implies "
            i+=1 #Porque este operador es de dos caracteres
        elif str_prop[i] == '<':
            if (i >= len(str_prop)-2) or str_prop[(i+1):(i+1+2)]!="->":
                print("String error 4")
                return None
            str2 += r"\Leftrightarrow " #r"\iff "
            i+=2 #Porque este operador es de tres caracteres
        #Caso especial: Cierre de paréntesis
        elif str_prop[i] == ')':
            str2 += ")"
        i+=1
    str2 += "$}"
    return str2

# Examples

Creating a file with the truth tables of the following propositions:
- $\neg a$
- $a\wedge b$
- $a\vee b$
- $a\implies b$
- $a\iff b$
- $\big((\neg q \wedge r )\vee p\vee q\big) \wedge \big(p\wedge r\big).$
- $\big(p\wedge\neg q\big) \wedge  \big(\neg(q\vee\neg p)\big)$
- $\big(q\vee r\vee s\big)\wedge \big(\neg(q\vee r)\big)\wedge \big(\neg(r\vee s)\big)\wedge \big(\neg(s\vee q)\big)$
- $\big(\neg(p\wedge q)\wedge\neg(p\wedge r)\big)\wedge\big( q\vee r\big)\wedge\big(\neg(p\implies \neg r)\big)$.

In [16]:
texTT("Demo.tex", ["¬a", "a^b", "a|b", "a->b", "a<->b", "(a<->b)->((c^a)<->(c^b))","((¬q^r)|p|q)^(p|r)", "(p^¬q)^(¬(q|¬p))","(q|r|s)^(¬(q|r))^(¬(r|s))^(¬(s|q))","(¬(p^q))^(¬(p^r))^(q|r)^(¬(p->¬r))"], standalone = True)

The file '/home/alexn/Documents/Proyectos/TruthTables/output/Demo.tex' has been created
